In [5]:
from random import randint
from sympy import isprime, mod_inverse

In [6]:
def Gen(q, g):
    if not isprime(q):
        raise ValueError("q debe ser un número primo.")
    alpha = randint(2, q - 1)
    u = pow(g, alpha, q)
    pk = u
    sk = alpha
    return pk, sk

In [7]:
q = 178286657344291
g = 10

pk, sk = Gen(q, g)

# Submensajes

In [8]:
def Enc(pk, m, q, g):
    if isinstance(m, int):
        m = str(m)

    # Determinar el tamaño de cada submensaje
    k = len(m) // 4
    if len(m) % 4 != 0:  # Ajustar si no es un múltiplo exacto de 4
        k += 1

    # Dividir el mensaje en submensajes
    submensajes = [m[i * 4: (i + 1) * 4] for i in range(k)]

    resultados = []
    for submensaje in submensajes:
        # Convertir submensaje a un número entero
        submensaje_int = int.from_bytes(submensaje.encode(), 'big')
        if not (1 < submensaje_int < q):
            raise ValueError(f"El valor entero del submensaje '{submensaje}' ({submensaje_int}) debe estar en el rango de 1 a q-1.")
        
        beta = randint(2, q - 1)
        v = pow(g, beta, q)
        c = (submensaje_int * pow(pk, beta, q)) % q
        resultados.append((beta, v, c))

    return resultados

In [9]:
# Mensaje a cifrar
m = '?Life Is Str@ge='

In [10]:
ci = []
# Cifrado del mensaje
try:
    resultados = Enc(pk, m, q, g)
    for i, (beta, v, c) in enumerate(resultados):
        print(f"(beta_{i}, v_{i}, c_{i}) = ({beta}, {v}, {c})")
        ci.append((v, c))
except ValueError as e:
    print(f"Error: {e}")

(beta_0, v_0, c_0) = (4728055643748, 123068799476540, 103983762358414)
(beta_1, v_1, c_1) = (141895486350214, 56349727799875, 158609655351354)
(beta_2, v_2, c_2) = (144404213368069, 71308206125449, 13738391959871)
(beta_3, v_3, c_3) = (12573070166586, 47267749568761, 152036262108765)


In [11]:
def Dec(sk, ciphers, q):
    message_parts = []

    for v, c in ciphers:
        w = pow(mod_inverse(v, q), sk, q)
        m_int = (c * w) % q
        try:
            # Calculamos el número de bytes a partir del valor máximo esperado en el cifrado
            num_bytes = max(1, (m_int.bit_length() + 7) // 8)
            m_bytes = m_int.to_bytes(num_bytes, byteorder='big')
            m_part = m_bytes.decode('utf-8', errors='ignore')  # Ignorar caracteres no decodificables
            message_parts.append(m_part)
        except (ValueError, UnicodeDecodeError, OverflowError) as e:
            message_parts.append(f"<cannot decode {m_int}: {str(e)}>")

    return ''.join(message_parts)

In [12]:
decrypted_message = Dec(sk, ci, q)

decrypted_message

'?Life Is Str@ge='

# Mensaje completo

In [13]:
def Gen(alpha, q, g):
    if not isprime(q):
        raise ValueError("q debe ser un número primo.")
    u = pow(g, alpha, q)
    pk = u
    sk = alpha
    return pk, sk

In [14]:
alpha = 44235728213098551077425602704492607016359920981613272052626024323091463300482880298541827967837430887576328472751910496011121592139764866532359692545249145346708241664924261327829449535530913639968563645084107647986106025250782392055385273661543061358700959794349801873828534525091734163432712627959693466322
p = 164502193028554071572773380673262121890970567103098787083558972687654868443151285432627053033053072000260424623411579165922995742644281638505768536442810062575979476066397674879391609594727231903485090135678243649110521956763797073399979304867490513217414015789052165354717083683136329248045988596838745639543
g = 95397605525288378204358698941767152376570780885616886203054870846705451384086811730296287946254767796802429470762715340386221298673197168854676481382299540380875518199684563702117074297547557764819044408561171097263130974134719766490187777579363041095603026258899214230765139858647550712580831085114196512019

In [15]:
pk, sk = Gen(alpha, p, g)
Gen(alpha, p, g)

(151158474933645961056109203843546757663499349704994753032344462176590045612277469424010507152938599881054324718700860339186411908384908868449443799062361609175755141536941336196209556949397507910519810299535179497103191237214847709868371078477624111558133485226442862166925545186872364562429628724456449964320,
 44235728213098551077425602704492607016359920981613272052626024323091463300482880298541827967837430887576328472751910496011121592139764866532359692545249145346708241664924261327829449535530913639968563645084107647986106025250782392055385273661543061358700959794349801873828534525091734163432712627959693466322)

In [16]:
def Enc(pk, m, q, g, beta):
    if isinstance(m, int):
        m = str(m)

    # Convertir el mensaje completo a un número entero
    m_int = int.from_bytes(m.encode(), 'big')
    if not (1 < m_int < q):
        raise ValueError(f"El valor entero del mensaje '{m}' ({m_int}) debe estar en el rango de 1 a q-1.")

    v = pow(g, beta, q)
    c = (m_int * pow(pk, beta, q)) % q

    return v, c

In [17]:
# Mensaje a cifrar
m = 'Life before death. Strength before weakness. Journey before destination.'

In [18]:
beta = 1674435797287954221334632204181841546464330290107923623942210552236484222083679006606991338204875162921537505044334242829398146213680515911546561003532481053511340400893917420383849309918369182487491981623929321347161779516300860326491830199554475284283107612032924929697179442226013402267076722291886004976

In [19]:
(v, c) = Enc(pk, m, p, g, beta)
print(f'v: {v}')
print(f'c: {c}')

v: 137324271447635478843236577542990771979838039824420694674021920547730751754425432321313309992680135519515605301538107016710226681685115428635282089812718284963444098237424833170677882254583336632211906092922856390084574044180218711378866290635852995765592960407861997390351515894469542331753318177126039703612
c: 152385699516201214510197269090045156436382590334250371152843210082653800689842726846861192029855516501633488476592129373700810649348816715973725825739986780641546606500453649319868883209011495566645862720015543862738854028605223425278648895645016205980082904785954296959241192245343234216618341378041326876886


In [20]:
def Dec(sk, vc, q):
    (v, c) = vc

    w = pow(mod_inverse(v, q), sk, q)
    m_int = (c * w) % q
    try:
        # Calculamos el número de bytes a partir del valor máximo esperado en el cifrado
        num_bytes = max(1, (m_int.bit_length() + 7) // 8)
        m_bytes = m_int.to_bytes(num_bytes, byteorder='big')
        m = m_bytes.decode('utf-8', errors='ignore')  # Ignorar caracteres no decodificables
    except (ValueError, UnicodeDecodeError, OverflowError) as e:
        m = (f"<cannot decode {m_int}: {str(e)}>")

    return m

In [21]:
v = 68061151581036716434892324158377054048188900523743874908698489105996914990262896768661881977846983773615022889533546058949562336174880904867521755870011084531901642697261435243761191875894067201992544525538048936053049390566477867608723507254180676340229106077392911507852355248583584948033867168598534756719
c = 69523011344786145101642032657815987921375318089731095568525610125234911341233953139836192784196094382143519342081211985332724352226158981239211946140210134621812415803764373047141297119036512929506081105143907943930942609377564301739840067655863765775883975776264774278146988355265292896498243587261096438860

In [22]:
Dec(sk, (v, c), p)

"There's always another secret."